In [3]:
import keras
import numpy as np
import pandas as pd
import cv2

In [4]:
track_1_log = pd.read_csv("track_1_data/driving_log.csv")
track_2_log = pd.read_csv("track_2_data/driving_log.csv")

In [5]:
i = 0
frames = []
for row in track_1_log.iterrows():
    input_frame = cv2.imread('track_1_data/' + row[1]['center'])
    frames.append(input_frame)

for row in track_2_log.iterrows():
    input_frame = cv2.imread('track_2_data/' + row[1]['center'])
    frames.append(input_frame)

In [6]:
X = frames
y = pd.concat([track_1_log.steering, track_2_log.steering])
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, Convolution2D, MaxPooling2D

model = Sequential()
model.add(Convolution2D(24, 5, 5, border_mode='same', input_shape=X_tr[0].shape))
model.add(Activation('relu'))
model.add(Convolution2D(36, 5, 5))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Convolution2D(48, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [8]:
from keras.preprocessing import image
datagen = image.ImageDataGenerator(
    rotation_range=0.03,
    width_shift_range=0.03,
    height_shift_range=0.03)

In [9]:
import gc
gc.collect()

4

In [ ]:
datagen.fit(X_tr)

# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(X_tr, y_tr, batch_size=32),
                    samples_per_epoch=len(X_tr), nb_epoch=32)